<a href="https://colab.research.google.com/github/NahinAlam001/499B/blob/UNet/UNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# ISIC-2017
!gdown 'https://drive.google.com/uc?export=download&id=1iwCXnS9u6OHD7ENFN76kCrZ7Uu98SUxv'

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1iwCXnS9u6OHD7ENFN76kCrZ7Uu98SUxv
From (redirected): https://drive.google.com/uc?export=download&id=1iwCXnS9u6OHD7ENFN76kCrZ7Uu98SUxv&confirm=t&uuid=106cb866-d47d-4ace-a590-eeda9a4981b6
To: /content/CSE499.zip
100% 13.0G/13.0G [04:17<00:00, 50.6MB/s]


In [2]:
import zipfile

with zipfile.ZipFile('CSE499.zip', 'r') as zip_ref:
  zip_ref.extractall()

In [3]:
!rm -rf sample_data
!rm CSE499.zip ISIC-2017_Test_v2_Part3_GroundTruth.csv ISIC-2017_Validation_Part3_GroundTruth.csv

##train.py

In [4]:
import os
import shutil

source_dirs = [
    "/content/ISIC-2017_Training_Data/ISIC-2017_Training_Data",
    "/content/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data",
    "/content/ISIC-2017_Validation_Data/ISIC-2017_Validation_Data"
]

destination_dir = "/content/isic-challenge-2017/ISIC2017_Task1-2_Training_Input"

if not os.path.isdir(destination_dir):
  os.makedirs(destination_dir, exist_ok=True)


def copy_jpg_files(source_dir, destination_dir):
    for root, _, files in os.walk(source_dir):
        subdirectory = os.path.relpath(root, source_dir)
        destination_subdirectory = os.path.join(destination_dir, subdirectory)
        os.makedirs(destination_subdirectory, exist_ok=True)

        for filename in files:
            if filename.lower().endswith(".jpg"):
                source_file = os.path.join(root, filename)
                destination_file = os.path.join(destination_subdirectory, filename)
                shutil.copy2(source_file, destination_file)


for source_dir in source_dirs:
    copy_jpg_files(source_dir, destination_dir)

print("JPG files copied successfully!")

JPG files copied successfully!


In [5]:
def walk_through_dir(path):
  for dirpath, _, files in os.walk(path):
    if len(files )!=0:
      print(f'There are {len(files)} in {dirpath}')
walk_through_dir(destination_dir)

There are 2750 in /content/isic-challenge-2017/ISIC2017_Task1-2_Training_Input


In [6]:
import os
import shutil

source_dirs = [
    "/content/ISIC-2017_Training_Part1_GroundTruth/ISIC-2017_Training_Part1_GroundTruth",
    "/content/ISIC-2017_Training_Part2_GroundTruth/ISIC-2017_Training_Part2_GroundTruth",
    "/content/ISIC-2017_Test_v2_Part1_GroundTruth/ISIC-2017_Test_v2_Part1_GroundTruth",
    "/content/ISIC-2017_Test_v2_Part2_GroundTruth/ISIC-2017_Test_v2_Part2_GroundTruth",
    "/content/ISIC-2017_Validation_Part1_GroundTruth/ISIC-2017_Validation_Part1_GroundTruth",
    "/content/ISIC-2017_Validation_Part2_GroundTruth/ISIC-2017_Validation_Part2_GroundTruth"
]

destination_dir = "/content/isic-challenge-2017/ISIC2017_Task1-2_Training_GroundTruth"

if not os.path.isdir(destination_dir):
  os.makedirs(destination_dir, exist_ok=True)


def copy_jpg_files(source_dir, destination_dir):
    for root, _, files in os.walk(source_dir):
        subdirectory = os.path.relpath(root, source_dir)
        destination_subdirectory = os.path.join(destination_dir, subdirectory)
        os.makedirs(destination_subdirectory, exist_ok=True)

        for filename in files:
            if filename.lower().endswith(".png"):
                source_file = os.path.join(root, filename)
                destination_file = os.path.join(destination_subdirectory, filename)
                shutil.copy2(source_file, destination_file)


for source_dir in source_dirs:
    copy_jpg_files(source_dir, destination_dir)

print("PNG files copied successfully!")

PNG files copied successfully!


In [7]:
walk_through_dir(destination_dir)

There are 2750 in /content/isic-challenge-2017/ISIC2017_Task1-2_Training_GroundTruth


In [9]:
import os
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from model import UNet

H, W = 256, 256

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

class ISICDataset(Dataset):
    def __init__(self, images, masks, transform=None):
        self.images = images
        self.masks = masks
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = cv2.imread(self.images[idx], cv2.IMREAD_COLOR)
        image = cv2.resize(image, (W, H)) / 255.0
        mask = cv2.imread(self.masks[idx], cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, (W, H)) / 255.0
        mask = np.expand_dims(mask, axis=-1)

        if self.transform:
            image = self.transform(image).float()
            mask = self.transform(mask).float()

        return image, mask

def load_data(dataset_path, split=0.2):
    images = sorted(glob(os.path.join(dataset_path, "ISIC2017_Task1-2_Training_Input", "*.jpg")))
    masks = sorted(glob(os.path.join(dataset_path, "ISIC2017_Task1-2_Training_GroundTruth", "*.png")))

    if not images or not masks:
        raise ValueError("No image or mask files found in the specified directories.")

    train_x, test_x = train_test_split(images, test_size=split, random_state=42)
    train_y, test_y = train_test_split(masks, test_size=split, random_state=42)
    train_x, valid_x = train_test_split(train_x, test_size=split, random_state=42)
    train_y, valid_y = train_test_split(train_y, test_size=split, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

if __name__ == "__main__":
    np.random.seed(42)
    torch.manual_seed(42)

    create_dir("files")

    batch_size = 4
    lr = 1e-4
    num_epochs = 10
    model_path = "/content/drive/MyDrive/499B/ISIC2017_new.pth"

    dataset_path = "/content/isic-challenge-2017"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

    transform = transforms.ToTensor()

    train_dataset = ISICDataset(train_x, train_y, transform=transform)
    valid_dataset = ISICDataset(valid_x, valid_y, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = UNet(in_channels=3, out_channels=1).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        valid_loss = 0
        with torch.no_grad():
            for images, masks in valid_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)
                loss = criterion(outputs, masks)
                valid_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(train_loader)}, Valid Loss: {valid_loss/len(valid_loader)}")

        torch.save(model.state_dict(), model_path)

##eval.py

In [14]:
!git clone -b UNet https://github.com/NahinAlam001/499B.git

import shutil

filenames = ["metrics.py", "train.py", "model.py"]

for filename in filenames:
  source = f"499B/{filename}"
  destination = filename
  shutil.copy2(source, destination)
  print(f"Copied {filename} from downloaded branch.")

fatal: destination path '499B' already exists and is not an empty directory.
Copied metrics.py from downloaded branch.
Copied train.py from downloaded branch.
Copied model.py from downloaded branch.


In [12]:
import os
import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import torch
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from metrics import dice_loss, dice_coef, iou
from train import load_data, create_dir
from model import UNet

H, W = 256, 256

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (H, W, 3)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x / 255.0
    x = x.astype(np.float32)
    x = np.transpose(x, (2, 0, 1))  ## Convert to (3, 256, 256)
    x = np.expand_dims(x, axis=0)   ## Add batch dimension
    x = torch.tensor(x, dtype=torch.float32)
    return ori_x, x  ## (1, 3, 256, 256)

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (H, W)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x / 255.0
    x = x.astype(np.int32)  ## (256, 256)
    x = torch.tensor(x, dtype=torch.int64)
    return ori_x, x

def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)  ## (256, 256, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1) ## (256, 256, 3)

    y_pred = np.expand_dims(y_pred, axis=-1)  ## (256, 256, 1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) ## (256, 256, 3)

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred * 255], axis=1)
    cv2.imwrite(save_image_path, cat_images)

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    torch.manual_seed(42)

    """ Folder for saving results """
    create_dir("results")

    """ Load the model """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = UNet(in_channels=3, out_channels=1).to(device)
    model.load_state_dict(torch.load("/content/drive/MyDrive/499B/ISIC2017.pth", map_location=device))
    model.eval()

    """ Load the test data """
    dataset_path = "/content/isic-challenge-2017"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

    SCORE = []
    with torch.no_grad():
        for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
            """ Extracting the image name """
            name = os.path.basename(x)

            """ Read the image and mask """
            ori_x, x = read_image(x)
            ori_y, y = read_mask(y)

            """ Predicting the mask """
            x = x.to(device)
            y_pred = model(x)
            y_pred = torch.sigmoid(y_pred)
            y_pred = (y_pred > 0.5).float()
            y_pred = y_pred.cpu().numpy()[0, 0]  ## Remove batch and channel dimensions
            y_pred = y_pred.astype(np.int32)

            """ Saving the predicted mask """
            save_image_path = f"results/{name}"
            save_results(ori_x, ori_y, y_pred, save_image_path)

            """ Flatten the array """
            y = y.numpy().flatten()
            y_pred = y_pred.flatten()

            """ Calculating metrics values """
            acc_value = accuracy_score(y, y_pred)
            f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
            jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
            recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
            precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
            SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

    """ mean metrics values """
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"\nAccuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    df = pd.DataFrame(SCORE, columns=["Image Name", "Acc", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("files/score.csv")

100%|██████████| 550/550 [03:22<00:00,  2.71it/s]

Accuracy: 0.20717
F1: 0.30500
Jaccard: 0.20717
Recall: 1.00000
Precision: 0.20717
